In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import torch
from torch.utils.data import Dataset
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## load data

In [ ]:
train_df =  pd.read_csv('/kaggle/input/hms-harmful-brain-activity-classification/train.csv')
for i, k in enumerate(train_df.columns):
    print(i,k)

In [ ]:
train_df.iloc[10,:]

## create a dataloader 

In [ ]:
import os
import pandas as pd
from torchvision.io import read_image

class HMSDatasetTrain(Dataset):
    def __init__(self, annotations_file, eeg_dir, train=True, transform=None, target_transform=None):
        self.df = pd.read_csv(annotations_file)
        self.eeg_dir = eeg_dir
        self.train = True
        self.transform = transform
        self.target_transform = target_transform
        #self.eeg_idx = self.df.columns.get_loc('eeg_id') # we do not have the same column index in train and in test

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        
        eeg_path = os.path.join(self.eeg_dir, str(self.df.iloc[idx, 0]))+'.parquet'
        # read the parquet file 
        eeg = pd.read_parquet(eeg_path)
        eeg = eeg.values
        offset = self.df.iloc[idx,2] # eeg_label_offset_seconds column
        start = int(offset)*200# the sample rate is 200
        duration = 50*200 # the duration is 
        eeg = eeg[start:start+duration,:]
        label = self.df.iloc[idx, 9:]
        label /=sum(label)
        label = label.to_dict()
        if self.transform:
            eeg = self.transform(eeg)

        if self.target_transform:
            label = self.target_transform(label)
        return eeg, label
        

annotations_file = '/kaggle/input/hms-harmful-brain-activity-classification/train.csv'
eeg_dir = '/kaggle/input/hms-harmful-brain-activity-classification/train_eegs'
training_data = HMSDatasetTrain(annotations_file,eeg_dir )
eeg, label = training_data[11]
eeg.shape, label

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(training_data, batch_size=2, shuffle=False)
next(iter(train_dataloader))

In [ ]:
train_df.head(2)